In [ ]:
!pip install langchain langchain-experimental langchain-openai --quiet

In [ ]:
!pip install langfuse --quiet

In [ ]:
!pip install --upgrade --quiet  yfinance langchain-community

In [ ]:
#AZURE Env variables
AZURE_OPENAI_API_KEY="04f9a983b5d747baac8c74a75c0d525a"
AZURE_OPENAI_ENDPOINT="https://swedencentral.api.cognitive.microsoft.com/"
OPENAI_API_VERSION="2024-02-15-preview"

In [ ]:
#Importing packages
from langchain_openai import AzureChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

from langchain.tools import Tool
from langchain_community.tools.yahoo_finance_news import YahooFinanceNewsTool
from langfuse.decorators import observe

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda

import yfinance as yf
import json

In [ ]:
model = AzureChatOpenAI(
    api_key=AZURE_OPENAI_API_KEY,
    api_version="2024-12-01-preview",
    azure_endpoint=AZURE_OPENAI_ENDPOINT,model='telcogpt'
    )

In [ ]:
from langfuse.openai import AzureOpenAI

client = AzureOpenAI(
    api_key=AZURE_OPENAI_API_KEY,
    api_version="2024-12-01-preview",
    azure_endpoint=AZURE_OPENAI_ENDPOINT
    )

In [ ]:
import os
os.environ["LANGFUSE_PUBLIC_KEY"] = "pk-lf-92dfa19a-b728-4b43-8463-ea1c65750959"
os.environ["LANGFUSE_SECRET_KEY"] = "sk-lf-bc7ec232-2199-410a-a875-8f4009d5a1c9"
os.environ["LANGFUSE_HOST"] = "https://cloud.langfuse.com"

In [ ]:
generate_prompt = ChatPromptTemplate.from_template("Enter company name (e.g., 'Apple Inc'): {task}")


In [ ]:
# Step 1: Accept Input (Prompt to accept Company Name)

@observe()
def generate_prompt(company_name: str)-> str:
    return input("Enter company name (e.g., 'Apple Inc'): ")

In [ ]:
# Step 2: Get Stock Code
# When full company name passed, this would return Company Symbol Code

@observe()
def get_stock_code(company_name: str) -> str:
        try:
            search = yf.Ticker(company_name)
            symbol = search.info.get("symbol")
            if not symbol:
                raise ValueError("Symbol not found in info.")
            return symbol
        except Exception:
            # Fallback: Static map
            static_map = {
                "Apple Inc": "AAPL",
                "Microsoft Corporation": "MSFT",
                "Tesla Inc": "TSLA",
                "Amazon.com Inc": "AMZN",
                "Oracle Corporation":"ORCL"
            }
            return static_map.get(company_name, "UNKNOWN")

In [ ]:
#Step 3: Fetch Company News using YahooFinanceNewsTool

@observe()
def fetch_news(ticker: str) -> str:
        yahoo_tool = YahooFinanceNewsTool()
        tool = Tool.from_function(
            func=yahoo_tool.run,
            name="YahooFinanceNews",
            description="Fetch recent news summaries, recent headlines and article summaries for this stock ticker in more than 200 words"
        )
        return tool.run(ticker)

In [ ]:
fetch_news("TSLA")

"Tesla Robotaxi Nearing Launch: Buy, Hold or Sell the Stock Now?\nTSLA jumps 23% in the past month on robotaxi hype, but with a few launch details and fading EV sales, is the rally built to last?\n\nMarket Chatter: Tesla Executives Troubled by Musk's Denial of $25,000 EV Cancellation\nTesla (TSLA) Chief Executive Elon Musk's denial last year of a Reuters report about the cancellation\n\nIs Trending Stock Tesla, Inc. (TSLA) a Buy Now?\nTesla (TSLA) has received quite a bit of attention from Zacks.com users lately. Therefore, it is wise to be aware of the facts that can impact the stock's prospects."

In [ ]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List

In [ ]:
# SentimentProfile Class to be used for parsing
class SentimentProfile(BaseModel):
    company_name: str = Field(...)
    stock_code: str = Field(...)
    newsdesc: str = Field(...)
    sentiment: str = Field(..., description="Positive, Negative, or Neutral")
    people_names: List[str]
    places_names: List[str]
    other_companies_referred: List[str]
    related_industries: List[str]
    market_implications: str
    confidence_score: float = Field(..., ge=0.0, le=1.0)

parser = PydanticOutputParser(pydantic_object=SentimentProfile)

In [ ]:
# Step 4: Function to analyze the Sentiment

def analyze_sentiment(news_summary: str):
        system_msg = "You are Analyst who are expert at financial news sentiment and entity analysis ."
        user_msg = f"""
Given the news headlines below for a company, analyze the overall sentiment Classify sentiment,
extract named entities: people, places, other companies and finally Provide a structured JSON with the following fields:
 company_name, stock_code, newsdesc, sentiment,Positive/Negative/Neutral, people_names,
  places_names, other_companies_referred, related_industries, market_implications,confidence_score

News:
{news_summary}

Respond in the following JSON format:
{parser.get_format_instructions()}
"""
        response = client.chat.completions.create(
            model="telcogpt",
            messages=[
                {"role": "system", "content": system_msg},
                {"role": "user", "content": user_msg}
            ],
            temperature=0.2,
        )

        content=response.choices[0].message.content
        return parser.parse(content)

In [ ]:
# Define chain
chain1 =  RunnableLambda(generate_prompt) | RunnableLambda(get_stock_code)|RunnableLambda(fetch_news)|RunnableLambda(analyze_sentiment)

In [33]:
# Invole Chain
op = chain1.invoke({"Company_Name"})
print(op)

Enter company name (e.g., 'Apple Inc'): Tesla Inc
company_name='Tesla, Inc.' stock_code='TSLA' newsdesc="Tesla Robotaxi Nearing Launch: Buy, Hold or Sell the Stock Now? TSLA jumps 23% in the past month on robotaxi hype, but with a few launch details and fading EV sales, is the rally built to last? Market Chatter: Tesla Executives Troubled by Musk's Denial of $25,000 EV Cancellation Tesla (TSLA) Chief Executive Elon Musk's denial last year of a Reuters report about the cancellation. Is Trending Stock Tesla, Inc. (TSLA) a Buy Now? Tesla (TSLA) has received quite a bit of attention from Zacks.com users lately. Therefore, it is wise to be aware of the facts that can impact the stock's prospects." sentiment='Mixed' people_names=['Elon Musk'] places_names=[] other_companies_referred=['Zacks.com', 'Reuters'] related_industries=['Automotive', 'Electric Vehicles'] market_implications="The news indicates a volatile sentiment around Tesla's stock, with potential for both positive and negative imp